In [0]:
jsonschema = '''
{
  "title": "products",
  "description": "This is a jsonschema file defining the table for validating a product record",
  "type": "object",
  "properties": {
    "productkey": {
      "description": "The unique identifier for the product",
      "type": "number"
    },
    "vendorname": {
      "description": "The name of the vendor",
      "type": "string"
    },
    "catalogid": {
      "description": "the id assigned by a vendor in a catalog",
      "type": "string"
    },
    "productdescription": {
      "description": "The description of the product",
      "type": "string"
    }
  },
  "required": [
    "vendorname",
    "catalogid",
    "productdescription"
  ],
  "additionalProperties": {
    "primarykeycolumn": "productkey",
    "uniquecolumns": [
      "vendorname",
      "catalogid",
      "productdescription"
    ]
  }
}'''

In [0]:
import json

class JsonSchemaTableCreator:
    def __init__(self, jsonschema):
        self.schema = json.loads(jsonschema)
        self.table_name = self.schema.get("title")
        self.properties = self.schema.get("properties", {})
        self.required = set(self.schema.get("required", []))
        self.additional = self.schema.get("additionalProperties", {})

    def spark_type(self, jsontype):
        mapping = {
            "string": "STRING",
            "number": "DOUBLE",
            "integer": "INT",
            "boolean": "BOOLEAN"
        }
        return mapping.get(jsontype, "STRING")

    def column_defs(self):
        cols = []
        for col, prop in self.properties.items():
            coltype = self.spark_type(prop.get("type"))
            notnull = "NOT NULL" if col in self.required else ""
            cols.append(f"{col} {coltype} {notnull}".strip())
        return ",\n  ".join(cols)

    def constraints(self):
        cons = []
        pk = self.additional.get("primarykeycolumn")
        if pk:
            cons.append(f"CONSTRAINT {self.table_name}_pk PRIMARY KEY ({pk})")
        uniq = self.additional.get("uniquecolumns")
        if uniq:
            cons.append(f"CONSTRAINT {self.table_name}_unique UNIQUE ({', '.join(uniq)})")
        return ",\n  ".join(cons)

    def create_table_sql(self):
        cols = self.column_defs()
        cons = self.constraints()
        sql = f"""
CREATE TABLE {self.table_name} (
  {cols}{',' if cons else ''}
  {cons}
)
USING DELTA
"""
        return sql

    def create_table(self):
        sql = self.create_table_sql()
        print(sql)
        # spark.sql(sql)


In [0]:
# Example usage:
creator = JsonSchemaTableCreator(jsonschema)
creator.create_table()

In [0]:


%sql

CREATE TABLE gold.product.products (
  productkey DOUBLE,
  vendorname STRING NOT NULL,
  catalogid STRING NOT NULL,
  productdescription STRING NOT NULL,
  CONSTRAINT products_pk PRIMARY KEY (productkey)--,
  --CONSTRAINT products_unique UNIQUE (vendorname, catalogid, productdescription)
)
USING DELTA